## ThirdAI's NeuralDB

NeuralDB, as the name suggests, is a combination of a neural network and a database. It provides a high-level API for users to insert different types of files into it and search through the file contents with natural language queries. The neural network part of it enables semantic search while the database part of it stores the paragraphs of the files that are inserted into it.

First, let's install the dependencies.

In [ ]:
!pip3 install thirdai --upgrade
!pip3 install "thirdai[neural_db]"
!pip3 install langchain --upgrade
!pip3 install openai --upgrade
!pip3 install paper-qa --upgrade

In [8]:
from thirdai import licensing, neural_db as ndb

import nltk
nltk.download("punkt")

import os
if "THIRDAI_KEY" in os.environ:
    licensing.activate(os.environ["THIRDAI_KEY"])
else:
    licensing.activate("")  # Enter your ThirdAI key here

[nltk_data] Downloading package punkt to /home/shubh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Now, let's import the relevant module and define a neural db class.

In [2]:
db = ndb.NeuralDB(user_id="my_user") # you can use any username, in the future, this username will let you push models to the model hub

### You even load from a base DB from our Bazaar (optional but recommended)

We have a model bazaar that provides users with domain specific NeuralDBs that can jumpstart searching on their private documents. The Bazaar has two main types of DBs

1. Base DBs: These come with models that have either general QnA capabilities or domain specific capabilities like search on Medical Documents, Financial documents or Contracts. These come with an empty data index into which users can insert their files.

2. Pre-Indexed DBs: These are ready-to-search DBs that come with pre-trained models and their corresponding datasets. These are meant to  search through large public datasets like PubMed or Amazon 3MM Products or Stackoverflow issues etc.

In [ ]:
# Set up a cache directory
import os
if not os.path.isdir("bazaar_cache"):
    os.mkdir("bazaar_cache")

from pathlib import Path
from thirdai.neural_db import Bazaar
bazaar = Bazaar(cache_dir=Path("bazaar_cache"))


Call fetch to refresh list of available DBs.

In [6]:
bazaar.fetch() # Optional arg filter="model name" to filter by model name.


Below is the list of all DBs in the Bazaar.

In [7]:
print(bazaar.list_model_names())


['Contract Review', 'Finance QnA', 'General QnA']


Finally load the DB

In [8]:
db = bazaar.get_model("General QnA")

### Insert your files

Let's insert things into it!

Currently, we natively support adding CSV, PDF and DOCX files. We also have a support to automatically scrape and parse URLs. All other file formats have to be converted into CSV files where each row represents a paragraph/text-chunk of the document. 

#### Example 1: CSV files
The first example below shows how to insert a CSV file. Please note that a CSV file is required to have a column named "DOC_ID" with rows numbered from 0 to n_rows-1.

In [14]:
insertable_docs = []
csv_files = ['data/sample_nda.csv']

for file in csv_files:
    csv_doc = ndb.CSV(
        path=file,
        id_column="DOC_ID",
        strong_columns=["passage"],
        weak_columns=["para"],  
        reference_columns=["passage"])
    #
    insertable_docs.append(csv_doc)


#### Example 2: PDF files

In [3]:
insertable_docs = []
pdf_files = ['data/sample_nda.pdf']

for file in pdf_files:
    pdf_doc = ndb.PDF(file)
    insertable_docs.append(pdf_doc)

#### Example 3: DOCX files

In [11]:
insertable_docs = []
doc_files = ['data/sample_nda.docx']

for file in doc_files:
    doc = ndb.DOCX(file)
    insertable_docs.append(doc)

#### Example 4: Parse from URLs directly

First you can use our utility to generate a set of candidate URLs containing data of interest. You can also use your own list of URLs to extract data from.
```python
valid_url_data = ndb.parsing_utils.recursive_url_scrape(
  base_url="https://www.thirdai.com/pocketllm/", max_crawl_depth=0
)
```
Then you can create a list of insertable documents from those URLs:
```python
insertable_docs = []
for url, response in valid_url_data:
    try:
        insertable_docs.append(ndb.URL(url, response))
    except:
        continue
```
These can be inserted into Neural DB just like any other document.
```python
db.insert(insertable_docs, ...)
```

### Insert into NeuralDB

If you wish to insert without unsupervised training, you can set 'train=False' in the insert() method.

In [ ]:
source_ids = db.insert(insertable_docs, train=False)

The above command is intended to be used with a base DB which already has reasonable knowledge of the domain. In general, we always recommend using 'train=True' as shown below.

#### Insert and Train

In [ ]:
source_ids = db.insert(insertable_docs, train=True)

If you call the insert() method multiple times, the documents will automatically be de-duplicated. If insert=True, then the training will be done multiple times.

### Search

Now let's start searching.

In [16]:
search_results = db.search(query="what is the termination period", top_k=2)

for result in search_results:
    print(result.text)
    # print(result.context(radius=1))
    # print(result.source)
    # print(result.metadata)
    print('************')

12. entire agreement. this agreement constitutes the entire agreement with respect to the subject matter hereof and supersedes all prior agreements and understandings between the parties (whether written or oral) relating to the subject matter and may not be amended or modified except in a writing signed by an authorized representative of both parties. the terms of this agreement relating to the confidentiality and non-use of confidential information shall continue after the termination of this agreement for a period of the longer of (i) five (5) years or (ii) when the confidential information no longer qualifies as a trade secret under applicable law.
************
4. return of confidential information. upon request of the other party termination of the discussions regarding a business relationship between the parties or termination of the current business relationship each party shall promptly destroy or deliver to the other party any and all documents notes and other physical embodim

We can see that the search pulled up the right passage that contains the termination period "(i) five (5) years or (ii) when the confidential information no longer qualifies as a trade secret" .

In [17]:
search_results = db.search(query="made by and between", top_k=2)

for result in search_results:
    print(result.text)
    # print(result.context(radius=1))
    # print(result.source)
    # print(result.metadata)
    print('************')

confidentiality agreement this confidentiality agreement (the “agreement”) is made by and between acme. dba tothemoon inc. with offices at 2025 guadalupe st. suite 260 austin tx 78705 and starwars dba tothemars with offices at the forest moon of endor and entered as of may 3 2023 (“effective date”).
************
in consideration of the business discussions disclosure of confidential information and any future business relationship between the parties it is hereby agreed as follows: 1. confidential information. for purposes of this agreement the term “confidential information” shall mean any information business plan concept idea know-how process technique program design formula algorithm or work-in-process request for proposal (rfp) or request for information (rfi) and any responses thereto engineering manufacturing marketing technical financial data or sales information or information regarding suppliers customers employees investors or business operations and other information or mat

We can see that the search pulled up the right passage again that has "made by and between".

Now let's ask a tricky question.

In [18]:
search_results = db.search(query="who are the parties involved?", top_k=2)

for result in search_results:
    print(result.text)
    # print(result.context(radius=1))
    # print(result.source)
    # print(result.metadata)
    print('************')

3. joint undertaking. each party agrees that it will not at any time disclose give or transmit in any manner or for any purpose the confidential information received from the other party to any person firm or corporation or use such confidential information for its own benefit or the benefit of anyone else or for any purpose other than to engage in discussions regarding a possible business relationship or the current business relationship involving both parties.
************
2. need to know. the receiving party shall limit its disclosure of the other party’s confidential information to those of its officers and employees and subcontractors (i) to which such disclosure is necessary for purposes of the discussions contemplated by this agreement and (ii) who have agreed in writing to be bound by provisions no less restrictive than those set forth in this agreement.
************


Oops! looks like when we search for "parties involved", we do not get the correct paragraph in the 1st position (we should be expecting the first paragraph as the correct results instead fo the last). 

No worries, we'll show shot to teach the model to correct it's retrieval.

### RLHF

Let's go over some of NeuralDB's advanced features. The first one is text-to-text association. This allows you to teach the model that two keywords, phrases, or concepts are related.

Based on the above example, let's teach the model that "parties involved" and the phrase "made by between" are the same.

In [19]:
db.associate(source="parties involved", target="made by and between")

Let's search again with the same query.

In [20]:
search_results = db.search(
    query="who are the parties involved?",
    top_k=2,
)

for result in search_results:
    print(result.text)
    # print(result.source)
    # print(result.metadata)
    print('************')

confidentiality agreement this confidentiality agreement (the “agreement”) is made by and between acme. dba tothemoon inc. with offices at 2025 guadalupe st. suite 260 austin tx 78705 and starwars dba tothemars with offices at the forest moon of endor and entered as of may 3 2023 (“effective date”).
************
in witness whereof this agreement has been duly executed by the parties hereto as of the latest date set forth below: acme inc. starwars inc. by: by: name: bugs bunny name: luke skywalker title: ceo title: ceo date: may 5 2023 date: may 7 2023
************


There you go! In just a line, you taught the model to correct itself and retrieve the correct result.

Now, let's see the 2nd option which is text-to-result association. Let's say that you know that "parties involved" should go the paragraph with DOC_ID=0, you can simply teach the model to associate the query to the corresponding label using the following API.

In [21]:
db.text_to_result("made by and between",0)

If you want to use the above RLHF methods in a batch instead of a single sample, you can simply use the batched versions of the APIs as shown next.

In [22]:
db.associate_batch([("parties involved","made by and between"),("date of signing","duly executed")])

In [23]:
db.text_to_result_batch([("parties involved",0),("date of signing",16)])

### Supervised Training (Optional)

If you have supervised data for a specific CSV file in your list, you can simply train the DB on that file by specifying a source_id = source_ids[*file_number_in_your_list*].

Note: The supervised file should have the query_column and id_column that you specify in the following call. The id_column should match the id_column that you specified in the "Prep CSV Data" step or default to "DOC_ID".

In [ ]:
sup_files = ['data/sample_nda_sup.csv']

db.supervised_train([ndb.Sup(path, query_column="QUERY", id_column="DOC_ID", source_id=source_ids[0]) for path in sup_files])

### Get Answers from OpenAI using Langchain

In this section, we will show how to use LangChain and query OpenAI's QnA module to generate an answer from the references that you retrieve from the above DB. You'll have to specify your own OpenAI key for this module to work. You can replace this segment with any other generative model of your choice. You can choose to use an source model like MPT or Dolly for answer generation with the same prompt that you use with OpenAI.

In [25]:
import os
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""  # Enter your OpenAI key here

In [26]:
from langchain.chat_models import ChatOpenAI
from paperqa.prompts import qa_prompt
from paperqa.chains import make_chain

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo', 
    temperature=0.1,
)

qa_chain = make_chain(prompt=qa_prompt, llm=llm)

In [27]:
def get_references(query):
    search_results = db.search(query,top_k=3)
    references = []
    for result in search_results:
        references.append(result.text)
    return references

def get_answer(query, references):
    return qa_chain.run(question=query, context='\n\n'.join(references[:3]), answer_length="abt 50 words")

In [ ]:
query = "what is the effective date of this agreement?"

references = get_references(query)
print(references)

In [29]:
answer = get_answer(query, references)

print(answer)

The effective date of this agreement is May 3, 2023 (Context).


### Get Embeddings

In [6]:
def get_embeddings(db, queries):
    return db._savable_state.model.model.embedding_representation([{"query":query} for query in queries])

In [9]:
embeddings = get_embeddings(db, ['query 1', 'query 2'])
print(embeddings.shape)
print(embeddings)


(2, 2048)
[[0.041311   0.07636996 0.03421707 ... 0.         0.         0.01327077]
 [0.04624894 0.04643891 0.00756899 ... 0.01081068 0.         0.0183362 ]]


### Load and Save
As usual, saving and loading the DB are one-liners.

In [ ]:
# save your db
db.save("sample_nda.ndb")

# Loading is just like we showed above, with an optional progress handler
db.from_checkpoint("sample_nda.ndb", on_progress=lambda fraction: print(f"{fraction}% done with loading."))

### Checkpointing Configuration for NeuralDB

To ensure the reliability of NeuralDB and facilitate easy recovery in case of failures, a straightforward checkpoint configuration is provided. This configuration allows users to define where and how often checkpoints are saved, and whether to resume from a checkpoint if needed.

In [2]:
db = ndb.NeuralDB(user_id="cautious_user")

doc_file = 'data/sample_nda.docx'
docs = [ndb.DOCX(doc_file)]

In [3]:
checkpoint_config = ndb.CheckpointConfig(
    checkpoint_dir = "./data/sample_checkpoint", # Specify the location for storing checkpoint data
    resume_from_checkpoint=False, # Set to True if you want to resume from a checkpoint
    checkpoint_interval=3 # Granularity of checkpoints (lower value implies more frequent checkpoints)
)

In [ ]:
def interrupt_training(percent_training_completed):
    # This function is used to stop the training in the middle
    if percent_training_completed > 0.5:
        raise StopIteration("Stopping Training")

try:
    # While inserting, we will stop the training in the middle.
    db.insert(
        sources = docs,
        on_progress = interrupt_training,
        checkpoint_config=checkpoint_config
    )
except StopIteration as ex:
    # Resuming from a saved checkpoint. Specifying resume_from_checkpoint as True will automatically load the 
    # checkpoint saved in the checkpoint directory specified in the Checkpoint Config. 
    checkpoint_config.resume_from_checkpoint=True
    db.insert(
        sources = docs,
        checkpoint_config=checkpoint_config
    )